In [17]:
import pandas as pd

In [18]:
data=pd.read_csv('Hotels_data_Changed.csv')

In [97]:
hotels=data.groupby(['Hotel Name']).size()
hotels=hotels.sort_values(ascending=False)
hotels=hotels.head(150)
hotelsNames=hotels.index
hotelsNames.values

array(['Newark Liberty International Airport Marriott',
       'Hilton Garden Inn Times Square', 'Westin New York at Times Square',
       'Loews Regency New York Hotel', 'Viceroy New York',
       'Four Seasons Hotel New York',
       'Residence Inn Newark Elizabeth Liberty International Airport',
       'Hilton Newark Airport',
       'Hilton Garden Inn New York-Times Square Central',
       'Hilton Garden Inn New York West 35th Street',
       'Le Parker Meridien New York',
       'DoubleTree by Hilton Metropolitan - New York City',
       'Hilton Garden Inn New York Midtown Park Ave',
       'The Lexington New York City Autograph Collection',
       'The Plaza Hotel', 'W New York - Union Square', 'Conrad New York',
       'Hyatt Union Square New York', 'Magnuson Convention Center Hotel',
       'Wyndham Garden Brooklyn Sunset Park', 'Cassa Hotel 45th Street',
       'DoubleTree by Hilton Hotel Newark Airport',
       'The Roosevelt Hotel New York City', 'Martha Washington',
       

In [100]:
checkinDates=data.loc[data['Hotel Name'].isin(hotelsNames)].groupby(['Checkin Date']).size()
checkinDates=checkinDates.sort_values(ascending=False)
checkinDates=checkinDates.head(40)
checkinDates=checkinDates.index
checkinDates.values

array(['2015-11-11', '2015-10-14', '2015-11-04', '2015-08-19',
       '2015-10-28', '2015-10-21', '2015-11-06', '2015-08-12',
       '2015-11-05', '2015-10-22', '2015-11-12', '2015-10-29',
       '2015-09-10', '2015-09-09', '2015-11-18', '2015-08-26',
       '2015-11-10', '2015-11-13', '2015-10-15', '2015-11-21',
       '2015-09-30', '2015-10-30', '2015-09-16', '2015-09-17',
       '2015-11-28', '2015-10-01', '2015-11-26', '2015-09-11',
       '2015-09-18', '2015-10-16', '2015-11-27', '2015-10-02',
       '2015-10-07', '2015-11-07', '2015-08-28', '2015-08-27',
       '2015-10-27', '2015-11-03', '2015-10-31', '2015-12-30'], dtype=object)

In [111]:
data=data.loc[data['Hotel Name'].isin(hotelsNames)].sort_values('Discount Price')
data=data[['Hotel Name','Checkin Date','Discount Code','Discount Price']]
data1=data

In [113]:
data.drop_duplicates(['Checkin Date','Discount Code','Hotel Name']).sort_values(['Checkin Date','Hotel Name'])

,Hotel Name,Checkin Date,Discount Code,Discount Price
181856,Andaz 5th Avenue - a concept by Hyatt,2015-08-12,3,1665
52147,Andaz Wall Street - a concept by Hyatt,2015-08-12,2,1325
42915,Andaz Wall Street - a concept by Hyatt,2015-08-12,3,1355
44445,Andaz Wall Street - a concept by Hyatt,2015-08-12,1,1365
122126,Best Western Plus Newark Airport West,2015-08-12,1,620
36849,Cassa Hotel 45th Street,2015-08-12,1,1335
33892,Cassa Hotel 45th Street,2015-08-12,2,1395
34235,Cassa Hotel 45th Street,2015-08-12,3,1515
43290,Cassa Hotel 45th Street,2015-08-12,4,1545
92817,Comfort Inn Times Square South Area,2015-08-12,1,1082


In [128]:
def foo(x,y):
        if x==0:
            return hotelsNames[y]
        else:
            return -1
initDate = [[foo(x,y) for x in range(160)] for y in range(149)]
print initDate

[['Newark Liberty International Airport Marriott', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], ['Hilton Garden Inn Times Square', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -